In [2]:
import fiona
import pandas as pd
import geopandas as gpd

In [19]:
from collections import OrderedDict
    
layers = fiona.listlayers('ipbes_pollination_summary_hg_2018-08-16_12_21_-0700_036a5f15f926.gpkg')

In [31]:
layers

['grid_1_degree',
 'pollination_table_full',
 'pollination_table',
 'total_ppl_poll',
 'pollination_table_with_negative_ones',
 'region_NCPsummary_positive',
 'region_NCPsummary_negative',
 'production_table',
 'region_PCsummary_positive',
 'region_PCsummary_negative',
 'c_poll_for_violins',
 'd_poll_and_tot_for_violins',
 'relevant_population',
 'c_unrealized',
 'UNxNCP',
 'UNxNCP_ssp1_incrpot',
 'UNxNCP_ssp3_incrpot',
 'UNxNCP_ssp5_incrpot',
 'UNxNCP_ssp1_incrpot_or',
 'UNxNCP_ssp3_incrpot_or',
 'UNxNCP_ssp5_incrpot_or',
 'region_summary_capped',
 'region_summary',
 'region_popimpacted_ssp1',
 'region_popimpacted_ssp3',
 'region_popimpacted_ssp5',
 'region_popveryimpacted_ssp1',
 'region_popveryimpacted_ssp3',
 'region_popveryimpacted_ssp5']

In [25]:
all_datasets = dict()

for layer in layers:
    all_datasets[layer.lower()] = gpd.read_file(
        "ipbes_pollination_summary_hg_2018-08-16_12_21_-0700_036a5f15f926.gpkg", layer=layer
    )

In [49]:
unxncp = all_datasets['pollination_table_full']

In [67]:
all_datasets['region_popveryimpacted_ssp1']

,region,sum(c_unrealized_avg_ssp1*relevant_min_pop_ssp1),sum(relevant_min_pop_ssp1),geometry
0,Africa,499665586966.89,173477750.039062,None
1,Eurasia,163593331.139757,33697123.0273437,None
2,North America,32754349.7100981,14524484.53125,None
3,North Asia,9371933.29546518,2288986.66015625,None
4,Oceania,71170439.3827369,7522617.10069275,None
5,South America,604481375.701584,13090557.6685181,None
6,South Asia,1173835398.91314,65954918.0410156,None


In [59]:
all_datasets['region_ncpsummary_positive']

,region,sum(gpw_v4_e_atot_pop_30s_ssp1),avg(poll_cont_nut_req_avg_1d_ssp1),avg(poll_cont_nut_req_avg_1d_cur),ssp,geometry
0,Africa,264935886.249634,0.0078357301146801,0.00574823577496112,1,None
1,Eurasia,502919110.283468,0.150478323794443,0.125907044476033,1,None
2,North America,117102587.112309,0.122349005227471,0.100559340999117,1,None
3,North Asia,912292969.250427,0.0607887526064785,0.0477730106643378,1,None
4,Oceania,7429192.77368152,0.0288722015731666,0.0188746282257921,1,None
5,South America,141162322.329681,0.0483222484710342,0.0376632328908291,1,None
6,South Asia,738799632.709076,0.0820569490823222,0.0599297988573974,1,None
7,Africa,344829508.779724,0.00324576585419886,0.00153550336549681,3,None
8,Eurasia,514902066.627968,0.148770163138772,0.131705566671786,3,None
9,North America,156970113.683462,0.087315901409609,0.0777009455114095,3,None


In [60]:
all_datasets['region_ncpsummary_negative']

,region,sum(gpw_v4_e_atot_pop_30s_ssp1),avg(poll_cont_nut_req_avg_1d_ssp1),avg(poll_cont_nut_req_avg_1d_cur),ssp,geometry
0,Africa,1439021362.16559,0.00319804723633231,0.00791461012568468,1,None
1,Eurasia,598587976.782608,0.101199163870687,0.120094534367543,1,None
2,North America,577215738.547634,0.0675933456684287,0.0810901190137401,1,None
3,North Asia,727252656.193237,0.0191795732191154,0.0215274056106456,1,None
4,Oceania,39084994.507164,0.0827666484461221,0.125831724605095,1,None
5,South America,297178178.215942,0.049457503357922,0.0525509521611607,1,None
6,South Asia,2084422582.21271,0.0371321461130696,0.0444257247332807,1,None
7,Africa,1859771497.75281,0.00321332328243652,0.0102071081761639,3,None
8,Eurasia,577002740.600105,0.0871358633536784,0.108595864456915,3,None
9,North America,519989301.388313,0.0706090536440789,0.0927674232312384,3,None


In [29]:
unxncp[unxncp['country'] == 'Chile']['GRIDCODE']

8000    38631
8065    38990
8066    38991
8067    38992
8130    39350
8131    39351
8132    39352
8192    39712
8301    40432
8302    40433
8353    40792
8445    41510
8446    41511
8484    41871
8485    41872
8523    42230
8524    42231
8568    42590
8569    42591
8617    42949
8618    42950
8619    42951
8669    43309
8670    43310
8671    43311
8719    43669
8720    43670
8771    44029
8772    44030
8773    44031
        ...  
9120    49067
9121    49068
9122    49069
9126    49426
9127    49427
9128    49428
9129    49429
9130    49787
9131    49788
9136    50505
9137    50506
9138    50508
9142    50867
9143    50868
9144    50869
9145    50870
9146    50871
9147    51227
9148    51228
9149    51229
9150    51230
9151    51231
9152    51232
9153    51588
9154    51589
9155    51590
9156    51591
9157    51592
9158    51951
9159    51952
Name: GRIDCODE, Length: 103, dtype: int64

Questions:

1. What does the GRIDCODE actually mean? and if they are the regions how can we relate them to latitude and longitude?
2. SSP1, SSP3, SSP5 are they the future scenario description? but how are these scenarios described?
3. Difference between semi natural and natural habitats?
4. 10s – globio, 30s – gpw, monfreda 2008 – 5m, ssp – 1/8d) clearing scales?
5. What does `c_` prefix in front of columns mean?

Observations:

- GRIDCODE is unique.